## Research Questions

### Economics questions

* How are the employees of the City of Chicago paid by job title relative to other areas with that job in the US?
  * We will need data at the level of the City of Chicago and at a national level, perhaps that supplied by the Bureau of Labor Statistics.
  

### Gender-related questions

If the dataset we start with doesn't have an indication of gender, how will we estimate it?

* How are men with the same job titles in the same departments paid relative to women in the City of Chicago?

* Do women do different jobs than men in the City?

* Are women or men in management roles proportional to the gender distribution of the department?
  * How do we define which roles are management roles?
  
### Safety-related questions

* What are the safest jobs in the City?
  * How do we define 'safe'? Where can we get data?

  
## Informational Views

* Show employees by department.
  * Enrichment idea: Merge in a description of department data so that we have a better idea of where people work.
* Show employees by job title.
* Show the salary of an individual by first or last name.

## We'll start with the "Name, Salary, Title" dataset from the Chicago Data Portal

https://data.cityofchicago.org/Administration-Finance/Current-Employee-Names-Salaries-and-Position-Title/xzkq-xp2w

> This dataset is a listing of all current City of Chicago employees, complete with full names, departments, positions, employment status (part-time or full-time), frequency of hourly employee –where applicable—and annual salaries or hourly rate. For hourly employees, the City is providing the hourly rate and frequency of hourly employees (40, 35, 20 and 10) to allow dataset users to estimate annual wages for hourly employees. Please note that annual wages will vary by employee, depending on number of hours worked and seasonal status. For information on the positions and related salaries detailed in the annual budgets, see https://www.cityofchicago.org/city/en/depts/obm.html


## Data Cleansing - Getting to Tidy Data

We'll need to explore this dataset to see what it offers. We can tell from the description that, at a minimum, we're going to need to do some work "to estimate annual wages for hourly employees". We want to be able to make comparisons of salary between groups for either of our research questions.

In [41]:
import seaborn as sns
sns.set()
import re

import pandas as pd

In [42]:
# Enable the notebook to automatically reload. 
# This makes it possible to move your code to the src folder and to use libraries you
# pip install along the way.

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
%matplotlib inline

In [44]:
data = pd.read_csv('../data/raw/chicago-salaries.csv')

In [45]:
data.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


In [46]:
data.describe(include='all')

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
count,33183,33183,33183,33183,33183,8022.000000,25161.000000,8022.000000
unique,32880,1111,35,2,2,NaN,NaN,NaN
top,"PEREZ, JOSE A",POLICE OFFICER,POLICE,F,Salary,NaN,NaN,NaN
freq,4,9520,13414,31090,25161,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,34.507604,86786.999790,32.788558
std,NaN,NaN,NaN,NaN,NaN,9.252077,21041.354602,12.112573
min,NaN,NaN,NaN,NaN,NaN,10.000000,7200.000000,2.650000
25%,NaN,NaN,NaN,NaN,NaN,20.000000,76266.000000,21.200000
50%,NaN,NaN,NaN,NaN,NaN,40.000000,90024.000000,35.600000
75%,NaN,NaN,NaN,NaN,NaN,40.000000,96060.000000,40.200000


## Tidy vs. Messy Data

When people talk about "cleaning" data, they really mean structuring the data in such a way as to make initial exploration and analysis easier. Instead of having to constantly re-invent the wheel and tailor your analysis workflow to the shape of every dataset, you reshape the data to fit a standard analysis workflow. Seaborn's plotting functions, for example, are optimized to work with [data in tidy format](https://seaborn.pydata.org/introduction.html#intro-tidy-data).

We want our data set to be "tidy".

Hadley Wickham coined the term [Tidy Data](https://vita.had.co.nz/papers/tidy-data.pdf) to describe data that is clean and ready for analysis. 

* Each variable is a column. A variable contains all values that measure the same underlying attribute (like height, temperature, duration) across units.
* Each observation is a row. An observation contains all values measured on the same unit (like a person, or a day, or a race) across attributes.
* Each type of observational unit is a table 

This is essentially [Codd's 3rd Normal Form](https://en.wikipedia.org/wiki/Third_normal_form) for a single dataset.

Messy data is any other arrangement of the data.

The five most common problems with messy datasets are:

* Column headers are values, not variable names. 
  * [Example](http://python-graph-gallery.com/boxplot/)
  * Solution: [`melt`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html) the data from wide to long format.
* Multiple variables are stored in one column. This can happen in either the column name or the column values.
  * Examples: 
    * `{'name': 'Shumway, Gordon'}`
    * `[{'sizeChicago': 42, 'sizeNewYork': 8}]`
  * Solution: Split the data so that each variable is a column.
    * `{'first_name': 'Gordon', 'last_name': 'Shumway'}`
    * `[{'size': 42, 'location': 'Chicago'}, {'size': 8, 'location': 'New York'}`
* Variables are stored in both rows and columns.
* Multiple types of observational units are stored in the same table.
  * Solution: Split the data into multiple tables; tidy those tables. Potentially merge them back into a single table.
* A single observational unit is stored in multiple tables.
  * Solution: Merge the data into a single table.


## Data Cleansing TODO List


**Rename columns to allow attribute access in Pandas.**

Pandas provides the ability to access columns directly with [attribute access](http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#attribute-access) indexing. For example, we can write `data.Department`, including auto-complete, instead of `data['Department']`. This is really a matter of convenience for us to work with data in Pandas more easily, but it's an easy win to take unless there's a good reason why we can't rename a column.

This only works if our attributes follow these rules:

* You can use this access only if the index element is a [valid Python identifier](https://docs.python.org/3/reference/lexical_analysis.html#identifiers), e.g. s.1 is not allowed.
* The attribute will not be available if it conflicts with an existing method name, e.g. `s.min` is not allowed.
* Similarly, the attribute will not be available if it conflicts with any of the following list: `index`, `major_axis`, `minor_axis`, `items`.
* In any of these cases, standard indexing will still work, e.g. `s['1']`, `s['min']`, and `s['index']` will access the corresponding element or column.

We can rename each of the columns to a valid Python identifier to make our life easier.


**Split first and last name.**

Tidy data means the data in each column can't be further split. Doing this with name may not seem to have any immediate benefits; treat it as a cheap speculative bet at this point.


**Handle NaN values for "Full or Part-Time" and "Salary or Hourly" observations.** 

This is an example of the "Multiple types of observational units are stored in the same table" type of mess.

We can see from the `head` preview above that there are `NaN` values. Some of the columns are only defined for Full-Time employees, and some only make sense for Part-Time. Full-time staff typically do different types of work than part-time. Some of part-time work is seasonal. Part-time staff are compensated differently. We'll need to do some work to compare the two types of employees.

There are four possible classes of the dataset:

<table>
    <tr><td>PT Hourly</td><td>PT Salary</td></tr>
    <tr><td>FT Hourly</td><td>FT Salary</td></tr>
</table>

Going back to our definition of tidy data, we want each row to contain all values measured on the same person across attributes. We want to compare groups of rows, not groups of columns, in our analysis. This means each column should have a meaningful value for every row.

The attributes which vary by class are "Typical Hours", "Annual Salary", and "Hourly Rate".

This means we need to ensure that every person has:

* `typical_hours` as 40 for full-time staff
* an `hourly_rate`, which is a function of salary / 52 / 40 for staff paid by salary.
* an `annualized_income`: We can get an approximation for this for hourly staff as a function of Typical Hours, Hourly Rate, and 52 weeks in a year.

**Investigate duplicates.**

We have 33183 entries, but only 32880 unique names. It seems there are ~300 duplicated names. This might be normal...some names are more common than others. We'll want to investigate to see if there's anything we need to do about this.

### Rename columns to allow attribute access in Pandas

In [47]:
# names = {'Name': 'name', 
#          'Job Titles': 'job_title',
#          'Department': ...
#         }

# data = data.rename(index=str, columns=names)

# OR...

# rename can also take a function to run on each of the columns, e.g.
# data.rename(re.sub("(\s|-)", "_", str.lower), axis='columns')

# you could define a function that will replace whitespace and dashes with underscores,
# then pass that function to rename.


In [48]:
import importlib
import src.data.cleaner as cln

importlib.reload(cln)

<module 'src.data.cleaner' from '/Users/ngraton/Desktop/code_platoon/w5/data_science/chicago_sal/src/data/cleaner.py'>

In [49]:
data = cln.pythonify_columns(data)

In [50]:
data.head()

,name,job_titles,department,full_or_part_time,salary_or_hourly,typical_hours,annual_salary,hourly_rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


In [51]:
new = data.name.str.split(",  ", n = 1, expand = True)

In [52]:
last_suf = new[0].str.split(" ", n = 1, expand = True)
first_midintl = new[1].str.split(" ", n = 1, expand = True)

In [53]:
mod_data = data
mod_data['first_name'] = first_midintl[0]
mod_data['middle_inital'] = first_midintl[1]
mod_data['last_name'] = last_suf[0]

mod_data['suffix'] = last_suf[1]
# mod_data = mod_data + data.drop(columns =["name"])

In [54]:
# for inital in mod_data.middle_inital:
#     inital = inital.strip()
#     if len(inital) > 1:

In [55]:
mod_data.head()

,name,job_titles,department,full_or_part_time,salary_or_hourly,typical_hours,annual_salary,hourly_rate,first_name,middle_inital,last_name,suffix
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN,JEFFERY,M,AARON,None
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN,KARINA,,AARON,None
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN,KIMBERLEI,R,AARON,None
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN,VICENTE,M,ABAD,JR
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86,REECE,E,ABASCAL,None


In [56]:
mod_data.middle_inital

0         M
1          
2         R
3         M
4         E
5          
6         J
7        L 
8         J
9         M
10        L
11         
12         
13        R
14         
15         
16        A
17        N
18        N
19         
20         
21         
22        V
23        S
24         
25        C
26        A
27         
28         
29        A
         ..
33153      
33154      
33155      
33156      
33157      
33158      
33159     M
33160      
33161      
33162    Z 
33163     E
33164      
33165     H
33166      
33167     M
33168     A
33169     J
33170     J
33171     W
33172      
33173      
33174     P
33175     J
33176      
33177      
33178      
33179     C
33180     E
33181     E
33182      
Name: middle_inital, Length: 33183, dtype: object

mod_data.head()